In [1]:
from aocd.models import Puzzle
import queue
puzzle = Puzzle(year=2023, day=20)
data = puzzle.input_data
%run helper.ipynb

In [2]:
class FF(object):
    def __init__(self, name, outputs):
        self.outputs = outputs
        self.state = False
        self.name = name
    
    def receive(self, signal):
        if not signal[1]:
            self.state = not self.state
            return [(self.name, self.state, output) for output in self.outputs]
        return []
        
class AND(object):
    def __init__(self, name, inputs, outputs):
        self.name = name
        self.state = {i: False for i in inputs}
        self.outputs = outputs
        
    def receive(self, signal):
        self.state[signal[0]] = signal[1]
        if all(self.state.values()):
            result = False
        else:
            result = True
        return [(self.name, result, output) for output in self.outputs]
    
def exhaust(processors, input_signals, i):
    iq = queue.Queue()
    for s in input_signals:
        iq.put(s)

    h, l = 0, 1
    hit_rx = False
    while not iq.empty():
        signal = iq.get(block=False)
        if signal[1]:
            h += 1
        else:
            l += 1
        if signal[2] == "rx":
            continue
        if signal[2] == "gf" and signal[1]:
            print(signal[0], i)
        results = processors[signal[2]].receive(signal)
        for result in results:
            iq.put(result)
    return h, l

def setup(data):
    processors = {}

    ps = [p.split(" -> ") for p in data.split("\n")]
    base_names = []
    for p in ps:
        if p[0] == "broadcaster":
            base_names = p[1].split(", ")
            continue
        if p[0][0] == "%":
            name = p[0][1:]
            processors[name] = FF(name, p[1].split(", "))
        if p[0][0] == "&":
            name = p[0][1:]
            inputs = [x[0][1:] for x in ps if name in x[1]]
            processors[name] = AND(name, inputs, p[1].split(", "))

    base_signals = [("b", False, name) for name in base_names]
    return processors, base_signals

In [3]:
h_total = 0
l_total = 0
processors, base_signals = setup(data)
for i in range(1000):
    h, l = exhaust(processors, base_signals, i)
    h_total += h
    l_total += l

puzzle.answer_a = h_total * l_total

In [4]:
processors, base_signals = setup(data)
i = 1
for i in range(1, 5000): # 5000 chosen to hit each LCM once
    _, _ = exhaust(processors, base_signals, i)

kr 3761
kf 3767
qk 4001
zs 4091


In [5]:
puzzle.answer_b = 231897990075517 # = LCM (3761, 3767, 4001, 4091)